# Kerchunk, GeoTIFF and Generating Coordinates with `xrefcoord`


<img src="../images/silo.png" width=400 alt="ARG"></img>

## Overview

In this tutorial we will cover:

1. How to generate `Kerchunk` references of GeoTIFFs.
1. Combining `Kerchunk` references into a virtual dataset.
1. Generating Coordinates with the `xrefcoord` accessor.


## Prerequisites
| Concepts | Importance | Notes |
| --- | --- | --- |
| [Kerchunk Basics](../foundations/kerchunk_basics) | Required | Core |
| [Multiple Files and Kerchunk](../foundations/kerchunk_multi_file) | Required | Core |
| [Kerchunk and Dask](../foundations/kerchunk_dask) | Required | Core |
| [Introduction to Xarray](https://foundations.projectpythia.org/core/xarray/xarray-intro.html) | Required | IO/Visualization |
- **Time to learn**: 30 minutes
---

## Motivation

GeoTIFF


## About the Dataset

SILO is an Australian climate dataset ranging from 1889-present. It contains daily and aggregated climate fields in both NetCDF and GeoTIFF formats. A version of it is hosted on the Registry of Open Data on AWS.

More details on this dataset can be found [here](https://registry.opendata.aws/silo/) and [here](https://www.longpaddock.qld.gov.au/silo/).


### Flags
In the section below, set the `subset` flag to be `True` (default) or `False` depending if you want this notebook to process the full file list. If set to `True`, then a subset of the file list will be processed (Recommended)

In [ ]:
subset_flag = True

In [ ]:
import glob
import logging
from tempfile import TemporaryDirectory

import dask
import fsspec
import numpy as np
# import xrefcoord  # noqa
import rioxarray
import s3fs
import ujson
import xarray as xr
from distributed import Client
from kerchunk.combine import MultiZarrToZarr
from kerchunk.tiff import tiff_to_zarr
from tqdm import tqdm

### Examining a Single GeoTIFF File

Before we use `Kerchunk` to create indices for multiple files, we can load a single GeoTiff file to examine it. 


In [ ]:
# URL pointing to a single NetCDF file
url = "s3://silo-open-data/Recommended/daily/daily_rain/2023/20230711.daily_rain.tif"

# Initialize a s3 filesystem
fs = s3fs.S3FileSystem(anon=True)

xds = rioxarray.open_rasterio(fs.open(url))

In [ ]:
xds

## Create Input File List

Here we are using `fsspec's` glob functionality along with the *`*`* wildcard operator and some string slicing to grab a list of GeoTIFF files from a `s3` `fsspec` filesystem. 

In [ ]:
# Initiate fsspec filesystems for reading
fs_read = fsspec.filesystem("s3", anon=True, skip_instance_cache=True)

file_paths = fs_read.glob("s3://silo-open-data/Recommended/daily/daily_rain/2023/*")

# Here we prepend the prefix 's3://', which points to AWS.
file_pattern = sorted(["s3://" + f for f in file_paths])


# If the subset_flag == True (default), the list of input files will be subset to speed up the processing
if subset_flag:
    file_pattern = file_pattern[0:6]

In [ ]:
file_pattern

In [ ]:
# This dictionary will be passed as kwargs to `fsspec`. For more details, check out the `foundations/kerchunk_basics` notebook.
so = dict(mode="rb", anon=True, default_fill_cache=False, default_cache_type="first")

# We are creating a temporary directory to store the .json reference files
# Alternately, you could write these to cloud storage.
td = TemporaryDirectory()
temp_dir = td.name
temp_dir

## Start a Dask Client

To parallelize the creation of our reference files, we will use `Dask`. For a detailed guide on how to use Dask and Kerchunk, see the Foundations notebook: [Kerchunk and Dask](../foundations/kerchunk_dask).


In [ ]:
client = Client(n_workers=8, silence_logs=logging.ERROR)
client

In [ ]:
# Use Kerchunk's `tiff_to_zarr` method to create create reference files


def generate_json_reference(fil, output_dir: str):
    tiff_chunks = tiff_to_zarr(fil, remote_options={"protocol": "s3"})
    fname = fil.split("/")[-2]
    outf = f"{output_dir}/{fname}.json"
    with open(outf, "wb") as f:
        f.write(ujson.dumps(tiff_chunks).encode())
    return outf


# Generate Dask Delayed objects
tasks = [dask.delayed(generate_json_reference)(fil, temp_dir) for fil in file_pattern]

In [ ]:
# Start parallel processing
import warnings

warnings.filterwarnings("ignore")
dask.compute(tasks)

## Combine Reference Files into Multi-File Reference Dataset

Now we will combine all the reference files generated into a single reference dataset. Since each TIFF file is a single timeslice and the only temporal information is stored in the filepath, we will have to specify the `coo_map` kwarg in `MultiZarrToZarr` to build a dimension from the filepath attributes. 



In [ ]:
# Custom Kerchunk function from `coo_map` to create dimensions
def fn_to_time(index, fs, var, fn):
    import datetime
    import re

    subst = fn.split("/")[-2].split("_")[2]
    return datetime.datetime.strptime(subst, "%Y%m%d")


mzz = MultiZarrToZarr(
    path=file_paths,
    indicts=sorted(glob.iglob(f"{temp_dir}/*.json")),
    remote_protocol="s3",
    coo_map={"time": fn_to_time},
    coo_dtypes={"time": np.dtype("M8[s]")},
    concat_dims=["time"],
    identical_dims=["x", "y"],
)

# save translate reference in memory for later visualization
multi_kerchunk = mzz.translate()

# Write kerchunk .json record
output_fname = "SILO.json"
with open(f"{output_fname}", "wb") as f:
    f.write(ujson.dumps(multi_kerchunk).encode())

## Open Combined Reference Dataset

In [ ]:
fs = fsspec.filesystem(
    "reference",
    fo="SILO.json",
    remote_protocol="s3",
    remote_options={"anon": True},
    skip_instance_cache=True,
)
m = fs.get_mapper("")
ds = xr.open_dataset(m, engine="zarr")

In [ ]:
ds

## Use `xrefcoord` to Generate Coordinates
When using `Kerchunk` to generate reference datasets for GeoTIFF's, only the dimensions are preserved. `xrefcoord` is a small utility that allows us to generate coordinates for these reference datasets using the geospatial metadata. Similar to other accessor add-on libraries for `Xarray` such as `rioxarray` and `xwrf`, `xrefcord` provides an accessor for an `Xarray` dataset. Importing `xrefcoord` allows us to use the `.xref` accessor to access additional methods. 

In the following cell, we will use the `generate_coords` method to build coordinates for the `Xarray` dataset.


In [ ]:
ref_ds = ds.xref.generate_coords(time_dim_name="time", x_dim_name="X", y_dim_name="Y")
ref_ds = ref_ds.rename({"0": "daily_rain"})

In [ ]:
ref_ds